## Bring in Alignment for mapping

This program will map TFBS using the Biopython's motif package.

**Inputs**: 
1. before alignment (fasta) 
2. after alignment (fasta) 
3. TFBS Position Frequency Matrix.

**Outputs**:
1. `.csv` file that outputs found TFBSs at each position, if any, in alignment (position, score, species, raw_position, strand motif_found)
2. `.csv` file that outputs only TFBS found.

**To Do**:

- [ ] **Fix bug where species are labelled wrong only in reverse strand (how is this possible)?**
- [ ] **Why are there NaNs when I pivot the table? Weird.**
- [x] Make Vector that attaches
- [ ] Attach real species name
- [ ] Loop through all files in directory
- [ ] Write file name in last column
- [ ] Append to a file 
- [ ] Have all of the important checks read out into a file

In [1]:
from Bio import motifs
from Bio import SeqIO 
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC, generic_dna, generic_protein
from collections import defaultdict
import re
import pandas as pd
import numpy as np
import os, sys

In [2]:
####################
## Functions
#####################

## Print full DF
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [3]:
#####################
## sys Inputs - to do
#####################

## So I can run in shell and loop through sequences AND motifs to get a giant dataset

### input1 = open(sys.argv[1]) #file that contains text
## input2 = open(sys.argv[2]) #file that contains terms to search for

In [4]:
##################
## Input 1 - Alignment Input 
##################

## read in alignment as a list of sequences
alignment = list(SeqIO.parse("../data/fasta/output_ludwig_eve-striped-2.fa", "fasta"))

## Sort alphabetically
alignment = [f for f in sorted(alignment, key=lambda x : x.id)]

## Make all ids lowercase

for record in alignment:
    record.id = record.id.lower()

## Check 
print("Found %i records in alignment file" % len(alignment))

## Sequence Length should be the same for all alignment sequences
for seq in alignment:
    print len(seq)
    
    
## Not being used.
## Turn sequences into a list of strings
## Note: They are no longer bio.seq.seq objects

## alignment_string_list = []
## for seq in alignment:
    ##alignment_string_list.append(str(seq.seq))


Found 9 records in alignment file
1136
1136
1136
1136
1136
1136
1136
1136
1136


In [5]:
#####################
## Input 2 - Raw Sequences Input
#####################

## This is used to search for TFBSs in sequences
## [ ] Maybe I have to convert from raw??

raw_sequences = list(SeqIO.parse("../data/fasta/ludwig_eve-striped-2.fasta", "fasta"))
print("Found %i records in raw sequence file" % len(raw_sequences))

## Sort alphabetically
raw_sequences = [f for f in sorted(raw_sequences, key=lambda x : x.id)]

raw_id = []
for seq in raw_sequences:
    raw_id.append(seq.id)
    print len(seq)
    
## make all IUPAC.IUPACUnambiguousDNA()
raw_sequences_2 = []
for seq in raw_sequences:
    raw_sequences_2.append(Seq(str(seq.seq), IUPAC.IUPACUnambiguousDNA()))

#print(raw_sequences)
#print(raw_sequences_2)

Found 9 records in raw sequence file
868
862
913
928
905
909
875
898
868


In [6]:
#####################
## Input 3 - Motif Input
#####################

motif = motifs.read(open("../data/PWM/transpose_fm/bcd_FlyReg.fm"),"pfm")
## motif.weblogo("mymotif.png")
print(motif.counts)
pwm = motif.counts.normalize(pseudocounts=0.0) # Doesn't change from pwm
pssm = pwm.log_odds()
print(pssm) # Why do I need log odds exactly?
motif_length = len(motif) #for later retrival of nucleotide sequence


        0      1      2      3      4      5      6      7
A:   0.19   0.17   0.88   0.92   0.04   0.04   0.06   0.12
C:   0.37   0.08   0.04   0.02   0.02   0.87   0.52   0.25
G:   0.08   0.04   0.04   0.04   0.33   0.02   0.08   0.37
T:   0.37   0.71   0.04   0.02   0.62   0.08   0.35   0.27

        0      1      2      3      4      5      6      7
A:  -0.38  -0.53   1.82   1.88  -2.70  -2.70  -2.12  -1.12
C:   0.55  -1.70  -2.70  -3.70  -3.70   1.79   1.05  -0.00
G:  -1.70  -2.70  -2.70  -2.70   0.39  -3.70  -1.70   0.55
T:   0.55   1.51  -2.70  -3.70   1.30  -1.70   0.47   0.11



In [7]:
######################
## Searching for Motifs in Sequences
######################

## Returns a list of arrays with a score for each position

## This give the score for each position
## If you print the length you get the length of the sequence minus TFBS length. 

## Forward stand

pssm_list = [ ]
for seq in raw_sequences_2:
    pssm_list.append(pssm.calculate(seq))

## Check
## You should notice that the length is total length of raw_sequence minus length of motif.
## for seq in pssm_list:
##    print len(seq)
    
## Reverse strand
## rpssm = pssm.reverse_complement()

In [8]:
########################## 
## Automatic Calculation of threshold
##########################

## Ideal to find something that automatically calculates, as
## opposed to having human choosing.

## Approximate calculation of appropriate thresholds for motif finding 
## Patser Threshold
## It selects such a threshold that the log(fpr)=-ic(M) 
## note: the actual patser software uses natural logarithms instead of log_2, so the numbers 
## are not directly comparable. 

distribution = pssm.distribution(background=motif.background, precision=10**4)
patser_threshold = distribution.threshold_patser() #for use later

print("Patser Threshold is %5.3f" % patser_threshold) # Calculates Paster threshold. 

Patser Threshold is 3.262


In [10]:
###################################
## Searching for motif in all raw_sequences
#################################

record_length = []
for record in raw_sequences_2:
    record_length.append(len(record))

position_list = []
for i in range(0,8):
    for position, score in pssm.search(raw_sequences_2[i], threshold = patser_threshold):
        positions = {'species': raw_id[i], 'score':score, 'position':position, 'seq_len': record_length[i] }
        position_list.append(positions)
        
position_DF = pd.DataFrame(position_list)

## Check
print position_DF

     position      score  seq_len                           species
0          10   5.454241      868  ludwig_eve-striped-2||memb002a|+
1         144  10.457056      868  ludwig_eve-striped-2||memb002a|+
2        -697   8.946094      868  ludwig_eve-striped-2||memb002a|+
3        -625   5.243600      868  ludwig_eve-striped-2||memb002a|+
4        -536   6.417715      868  ludwig_eve-striped-2||memb002a|+
5         336   3.702168      868  ludwig_eve-striped-2||memb002a|+
6         377   3.491528      868  ludwig_eve-striped-2||memb002a|+
7        -469   3.414712      868  ludwig_eve-striped-2||memb002a|+
8        -461   3.296591      868  ludwig_eve-striped-2||memb002a|+
9         423   3.794091      868  ludwig_eve-striped-2||memb002a|+
10       -416   9.909568      868  ludwig_eve-striped-2||memb002a|+
11        455   4.094957      868  ludwig_eve-striped-2||memb002a|+
12       -404   5.787577      868  ludwig_eve-striped-2||memb002a|+
13        521   3.793224      868  ludwig_eve-st

In [11]:
#############################
## Add strand and pos position information as columns to position_DF
##############################

## raw_position column
## change position to positive
## [ ] Fix length problem
    
position_list_pos = []

for i, x in enumerate(position_DF['position']):
    if x < 0:
       position_list_pos.append(position_DF.loc[position_DF.index[i], 'seq_len'] + x)
    else:
       position_list_pos.append(x)

## append to position_DF
position_DF['raw_position'] = position_list_pos

## print position_DF['raw_position']
    
## strand Column
strand = []
for x in position_DF['position']:
    if x < 0:
       strand.append("negative")
    else:
       strand.append("positive")
    
## append to position_DF
position_DF['strand'] = strand

## motif_found column
## First turn into a list of strings
raw_sequences_2_list = []
for seq in raw_sequences_2:
    raw_sequences_2_list.append(str(seq))

## Now get all motifs found in sequences
# motif_found = []
# for x in position_DF['position']:
    # motif_found.append(raw_sequences_2_list[i][x:x + motif_length])

## append to position_DF
# position_DF['motif_found'] = motif_found

print position_DF

## Check
## len(motif_found)    
## print(motif_found) 
## print(position_DF)

     position      score  seq_len                           species  \
0          10   5.454241      868  ludwig_eve-striped-2||memb002a|+   
1         144  10.457056      868  ludwig_eve-striped-2||memb002a|+   
2        -697   8.946094      868  ludwig_eve-striped-2||memb002a|+   
3        -625   5.243600      868  ludwig_eve-striped-2||memb002a|+   
4        -536   6.417715      868  ludwig_eve-striped-2||memb002a|+   
5         336   3.702168      868  ludwig_eve-striped-2||memb002a|+   
6         377   3.491528      868  ludwig_eve-striped-2||memb002a|+   
7        -469   3.414712      868  ludwig_eve-striped-2||memb002a|+   
8        -461   3.296591      868  ludwig_eve-striped-2||memb002a|+   
9         423   3.794091      868  ludwig_eve-striped-2||memb002a|+   
10       -416   9.909568      868  ludwig_eve-striped-2||memb002a|+   
11        455   4.094957      868  ludwig_eve-striped-2||memb002a|+   
12       -404   5.787577      868  ludwig_eve-striped-2||memb002a|+   
13    

In [13]:
##################
## get alignment position 
#################

## Need to map to the sequence alignment position

remap_list = []
nuc_list = ['A', 'a', 'G', 'g', 'C', 'c', 'T', 't', 'N', 'n']


positions = {'score':score, 'position':position, 'species': i}
position_list.append(positions)

for i in range(0,9):
    counter = 0
    for xInd, x in enumerate(alignment[i].seq):    
        if x in nuc_list:
            remaps = {'raw_position': counter, 'align_position':xInd, 'species':alignment[i].id}
            counter += 1
            remap_list.append(remaps)
            
remap_DF = pd.DataFrame(remap_list)

## Check
## print_full(remap_DF)

      align_position  raw_position                           species
0                  0             0  ludwig_eve-striped-2||memb002a|+
1                  1             1  ludwig_eve-striped-2||memb002a|+
2                  2             2  ludwig_eve-striped-2||memb002a|+
3                  3             3  ludwig_eve-striped-2||memb002a|+
4                  4             4  ludwig_eve-striped-2||memb002a|+
5                  5             5  ludwig_eve-striped-2||memb002a|+
6                  6             6  ludwig_eve-striped-2||memb002a|+
7                  7             7  ludwig_eve-striped-2||memb002a|+
8                  8             8  ludwig_eve-striped-2||memb002a|+
9                  9             9  ludwig_eve-striped-2||memb002a|+
10                10            10  ludwig_eve-striped-2||memb002a|+
11                11            11  ludwig_eve-striped-2||memb002a|+
12                12            12  ludwig_eve-striped-2||memb002a|+
13                13            13

In [14]:
## A way to check sequences
## [ ] Write better test

for seq in alignment:
    print(seq.seq[345:353])
    
for seq in raw_sequences_2_list:
    print(seq[262:270])

TTATTGCC
TTATTGCC
TTATTGCC
TTATTGCC
TTATTGCC
TTATTGCC
TTATTGCC
TTATTGCC
TTATTGCC
ttgaacaa
ttattgcc
agatttta
cttgaaca
aacaatcg
ttgccgca
ttattgcc
cgagttag
gtgctcta


In [15]:
## Merge both datasets

## Check first
## print(position_DF.shape)
## print(remap_DF.shape)

## Merge - all sites
TFBS_map_DF_all = pd.merge(position_DF, remap_DF, on=['species', 'raw_position'], how='outer')

## Sort
TFBS_map_DF_all = TFBS_map_DF_all.sort_values(by=['species','align_position'], ascending=[True, True])

## Check
## print_full(TFBS_map_DF_all)
## print(TFBS_map_DF_all.shape)

# Merge - only signal 
TFBS_map_DF_only_signal = pd.merge(position_DF, remap_DF, on=['species', 'raw_position'], how='inner')
TFBS_map_DF_only_signal = TFBS_map_DF_only_signal.sort_values(by=['species','align_position'], ascending=[True, True])

## To quickly check if species share similar TFBS positions
## print_full(TFBS_map_DF_only_signal.sort_values(by=['raw_position'], ascending=[True]))


## Check
## print_full(TFBS_map_DF_only_signal)
## print(TFBS_map_DF_only_signal.shape)


print_full(TFBS_map_DF_all)

## Write out Files
TFBS_map_DF_all.to_csv('../data/outputs/TFBS_map_DF_all_bicoid_test.csv', sep='\t', na_rep="NA")

      position      score  seq_len                           species  \
234        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
235        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
236        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
237        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
238        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
239        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
240        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
241        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
242        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
243        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
0           10   5.454241      868  ludwig_eve-striped-2||memb002a|+   
244        NaN        NaN      NaN  ludwig_eve-striped-2||memb002a|+   
245        NaN        NaN      NaN  ludwig_eve-striped-2||memb00

In [16]:
###############################
## Print binary info for each species
################################

# Create new column, 1 if TFBS is present, 0 if absent
TFBS_map_DF_all['presence'] = 0 # For some reason you have to initaite the column first.
TFBS_map_DF_all['presence'] = TFBS_map_DF_all.apply(lambda x: x.notnull(), axis=1)
TFBS_map_DF_all['presence'] = TFBS_map_DF_all['presence'].astype(int)

## Check   
## print_full(TFBS_map_DF_all)

## Create new dataframe

## Check First
## list(TFBS_map_DF_all.columns)

TFBS_map_DF_binary = TFBS_map_DF_all[['species', 'presence', 'align_position', 'strand']].copy()

## Subset on strand

TFBS_map_DF_binary_positive = TFBS_map_DF_binary['strand'] == "positive"
TFBS_map_DF_binary_negative = TFBS_map_DF_binary['strand'] == "negative"

## Check
## print_full(TFBS_map_DF_binary)

## Now long to wide
## [ ] NaNs are introduced here and not sure why and It is super annoying.
## - Maybe it has something to do with the negative and positive strands. These should be subsetted first. 

TFBS_map_DF_binary = TFBS_map_DF_binary.pivot_table(index='species', columns='align_position', values='presence')

print_full(TFBS_map_DF_binary.iloc[:,6:15])

align_position                    6   7   8   9   10  11  12  13  14
species                                                             
ludwig_eve-striped-2||memb002a|+   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb002c|+   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb002d|+   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb002e|-   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb002f|+   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb003b|+   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb003c|-   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb003d|-   0   0   0   0   1   0   0   0   0
ludwig_eve-striped-2||memb003f|+   0   0   0   0   0   0   0   0   0


In [ ]:
####################
## Attach input files name as a column
#####################

## Ideally I would attach the file name of the 1. raw sequence and 2. the motif being tested


